<a href="https://colab.research.google.com/github/emdifiore22/Unsupervised-Learning-with-Autoencoder-for-Predictive-Maintenance/blob/main/DCASE2020_CONDITIONING_ID_10_FRAME_BEST_RESULTS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import necessari
import librosa
import numpy
import sys
import os
import glob
import itertools
import re
import pickle
import tensorflow as tf
import tensorflow.keras.models
import tensorflow.keras.backend as K
import keras.optimizers
from keras.models import Model
from keras.layers import Input, Dense, BatchNormalization, Activation, Flatten, Multiply, Add, Reshape
from tqdm import tqdm
from itertools import groupby
from keras.utils import to_categorical
from sklearn import metrics
from google.colab import drive

In [2]:
# costanti 
ALPHA = 0.75
N_MELS = 128
HOP_LENGTH = 512
N_FFT = 1024
POWER = 2.0
FRAME_NUMS = 313
NUM_FILES = 3349
FRAMES = 10
drive.mount('/content/drive')

Mounted at /content/drive


# Feature Extraction

In [3]:
# load dataset
def select_dirs(path):
    dir_path = os.path.abspath(path)
    dirs = sorted(glob.glob(dir_path))
    return dirs

def file_load(wav_name, mono=False):
    try:
        return librosa.load(wav_name, sr=None, mono=mono)
    except:
        logger.error("file_broken or not exists!! : {}".format(wav_name))

def file_list_generator(target_dir, dir_name="train", ext="wav"):
    print("target_dir : {}".format(target_dir))

    # generate training list
    training_list_path = os.path.abspath("{dir}/{dir_name}/*.{ext}".format(dir=target_dir, dir_name=dir_name, ext=ext))
    files = sorted(glob.glob(training_list_path))
    if len(files) == 0:
      print("errore")
    return files


def file_to_vector_array(file_name, n_mels=64, n_fft=1024, hop_length=512, power=2.0):
    # 02 generate melspectrogram using librosa
    y, sr = file_load(file_name)
    mel_spectrogram = librosa.feature.melspectrogram(y=y, sr=sr, n_fft=n_fft, hop_length=hop_length, n_mels=n_mels, power=power)

    # 03 convert melspectrogram to log mel energy
    log_mel_spectrogram = 20.0 / power * numpy.log10(mel_spectrogram + sys.float_info.epsilon)

    return log_mel_spectrogram

  
def list_to_vector_array(file_list, msg="calc...", n_mels=64, n_fft=1024, hop_length=512, power=2.0, frames=10):
    # iterate file_to_vector_array()
    for idx in tqdm(range(len(file_list)), desc=msg):
        vector_array = file_to_vector_array(file_list[idx], n_mels=n_mels, n_fft=n_fft, hop_length=hop_length, power=power)
       
        # vector_array = numpy.delete(vector_array,[310,311,312], axis=1)
        # vector_array = numpy.asarray(numpy.hsplit(vector_array, 31))

        if idx == 0:
            dataset = numpy.zeros((len(file_list), n_mels, frames), float)
        dataset[vector_array.shape[0] * idx: vector_array.shape[0] * (idx + 1), :] = vector_array
    return dataset

def key_by_id(item):
  path_splitted = item.split("/")
  file_name = path_splitted[ len(path_splitted) - 1 ]
  file_name_splitted = file_name.split("_")
  machine_id = file_name_splitted = file_name_splitted[2]
  return machine_id

In [4]:
# GENERAZIONE DEI DATI DI TRAINING
dirs = "/content/drive/MyDrive/dcase/pump/"
files = file_list_generator(dirs)

# La seguente riga estrae i dati dai file audio
# train_data = list_to_vector_array(files, msg="generate train_dataset", n_mels=N_MELS, n_fft=N_FFT, hop_length=HOP_LENGTH, power=POWER, frames=313)

# Input 1 (3349, 128, 313): matrici 128x313
#   spettrogrammi estratti dai segnali audio
# Loading da Google Drive
train_data = numpy.load("/content/drive/MyDrive/training_not_norm_05_def.npy")

target_dir : /content/drive/MyDrive/dcase/pump/


In [5]:
# GENERAZIONE DELLE LABELS

# load base_directory list
grouped_list_by_machine_id = [list(v) for k,v in groupby(sorted(files), key_by_id)]

# Print summary of the reading
print("\n\n")
print("Number of machine ID of type {tipo}: {num_id}".format(tipo = dirs, num_id = len(grouped_list_by_machine_id)))
print("======================")
for i in range(0, len(grouped_list_by_machine_id)):
  print("Number of instances with ID {id}: {num_ist}".format(id=i, num_ist=len(grouped_list_by_machine_id[i])))
print("======================")

# One-hot encoding
label = []
choices = []
for i in range(0, len(grouped_list_by_machine_id)):
  for j in range(0, len(grouped_list_by_machine_id[i])):
    machine_id = grouped_list_by_machine_id[i][j].split('/')[7].split('_')[2]
    #print(grouped_list_by_machine_id[i][j].split('/')[7])
    random_choice = numpy.random.choice(["match", "non_match"], p = [ALPHA, 1-ALPHA]) 

    if machine_id == '00':
      if random_choice == "match":
        to_append = [1,0,0,0]
      else: 
        not_match_label = numpy.random.choice([1, 2, 3]) 
        if not_match_label == 1:
          to_append = [0,1,0,0]
        elif not_match_label == 2:
          to_append = [0,0,1,0]
        else: 
          to_append = [0,0,0,1]

    elif machine_id == '02': 

      if random_choice == "match":
        to_append = [0,1,0,0]
      else: 
        not_match_label = numpy.random.choice( [ 1, 2, 3] ) 
        if not_match_label == 1:
          to_append = [1,0,0,0]
        elif not_match_label == 2:
          to_append = [0,0,1,0]
        else: 
          to_append = [0,0,0,1]

    elif machine_id == "04":
      
      if random_choice == "match":
        to_append = [0,0,1,0]
      else: 
        not_match_label = numpy.random.choice( [ 1, 2, 3] ) 
        if not_match_label == 1:
          to_append = [1,0,0,0]
        elif not_match_label == 2:
          to_append = [0,1,0,0]
        else: 
          to_append = [0,0,0,1]

    elif machine_id == "06":
      if random_choice == "match":
        to_append = [0,0,0,1]
      else: 
        not_match_label = numpy.random.choice( [ 1, 2, 3] ) 
        if not_match_label == 1:
          to_append = [1,0,0,0]
        elif not_match_label == 2:
          to_append = [0,1,0,0]
        else: 
          to_append = [0,0,1,0]
    
    label.append(to_append) # Append della label associata a ciascuno spettrogramma
    choices.append(random_choice) # Append della choice utilizzata per associare la label
                                  # La choice sarà utile in fase di addestramento per capire che tipo di loss calcolare

# Trasformazione in numpy.array     
label_training = numpy.asarray(label)
choices_training = numpy.asarray(choices)
print(label_training.shape)
print(choices_training.shape)




Number of machine ID of type /content/drive/MyDrive/dcase/pump/: 4
Number of instances with ID 0: 906
Number of instances with ID 1: 905
Number of instances with ID 2: 602
Number of instances with ID 3: 936
(3349, 4)
(3349,)


In [6]:
# Estrazione spettrogrammi divisi per ID
id_00 = train_data[0:906]
id_02 = train_data[906:1811]
id_04 = train_data[1811:2413]
id_06 = train_data[2413:3349]


# MIN-MAX Normalization per ID_00
id_00_norm = numpy.empty_like(id_00)
max_00 = numpy.max(id_00)
min_00 = numpy.min(id_00)
index = 0
for elem in id_00:
  elem = ( elem - min_00 ) / ( max_00 - min_00)
  id_00_norm[index] = elem
  index = index + 1 

# MIN-MAX Normalization per ID_02
id_02_norm = numpy.empty_like(id_02)
max_02 = numpy.max(id_02)
min_02 = numpy.min(id_02)
index = 0
for elem in id_02:
  elem = ( elem - min_02 )/(max_02-min_02)
  id_02_norm[index] = elem
  index = index + 1 

# MIN-MAX Normalization per ID_04
id_04_norm = numpy.empty_like(id_04)
max_04 = numpy.max(id_04)
min_04 = numpy.min(id_04)
index = 0
for elem in id_04:
  elem = (elem-min_04)/(max_04-min_04)
  id_04_norm[index] = elem
  index = index + 1 

# MIN-MAX Normalization per ID_06
id_06_norm = numpy.empty_like(id_06)
max_06 = numpy.max(id_06)
min_06 = numpy.min(id_06)
index = 0
for elem in id_06:
  elem = ( elem - min_06 ) / ( max_06 - min_06 )
  id_06_norm[index] = elem
  index = index + 1 

print(id_00_norm.shape)
print(id_02_norm.shape)
print(id_04_norm.shape)
print(id_06_norm.shape)

# Ricreazione di un unico dataset contenente i vari dati opportunamente normalizzati
train_data_norm = numpy.concatenate([id_00_norm, id_02_norm, id_04_norm, id_06_norm])

print(train_data_norm.shape)

(906, 128, 313)
(905, 128, 313)
(602, 128, 313)
(936, 128, 313)
(3349, 128, 313)


In [7]:
# Questi minimi e massimi saranno utilizzati in fase di testing.

# Print dei minimi per ciascun ID.
print(min_00)
print(min_02)
print(min_04)
print(min_06)

# Print dei massimi per ciascun ID.
print(max_00)
print(max_02)
print(max_04)
print(max_06)

-61.276222229003906
-57.22140884399414
-60.722129821777344
-68.19204711914062
5.679316520690918
7.576060771942139
7.363907814025879
8.988851547241211


In [8]:
# Estrazione frame 128x10 da ciascun spettrogramma
training = numpy.zeros((200940, 128, 10)) # Dataset utilizzato per il training
index = 0
for vector_array in train_data_norm:
  i = 0
  while (i+5) < 303:
    vector_i = numpy.zeros((128,10))
    for j in range(0,128):
      vector_i[j] = vector_array[j][i:i+10]
    training[index] = vector_i
    index += 1
    i = i+5

# Associazione della label associata a ciascun spettrogramma a ciascuno dei frame estratto da esso.
training_labels = []
for elem in label_training:
  if numpy.array_equal(elem, numpy.asarray([1,0,0,0])) :
    for i in range(60):
      training_labels.append([1,0,0,0])
  elif numpy.array_equal(elem, numpy.asarray([0,1,0,0])):
    for i in range(60):
      training_labels.append([0,1,0,0])
  elif numpy.array_equal(elem, numpy.asarray([0,0,1,0])):
    for i in range(60):
      training_labels.append([0,0,1,0])
  elif numpy.array_equal(elem, numpy.asarray([0,0,0,1])):
    for i in range(60):
      training_labels.append([0,0,0,1])

# Associazione della choice associata a ciascun spettrogramma a ciascuno dei frame estratto da esso. 
training_choices = []
for elem in choices_training:
  if numpy.array_equal(elem, numpy.asarray("match")) :
    for i in range(60):
      training_choices.append("match")
  elif numpy.array_equal(elem, numpy.asarray("non_match")):
    for i in range(60):
      training_choices.append("non_match")

training_labels = numpy.asarray(training_labels) # Dataset utilizzato per il training
training_choices = numpy.asarray(training_choices) # Dataset utilizzato per il training

In [9]:
print(training.shape)
print(training_labels.shape)
print(training_choices.shape)

(200940, 128, 10)
(200940, 4)
(200940,)


In [10]:
# Shuffling
randomize = numpy.arange(len(training))
numpy.random.shuffle(randomize)
training_tot_shuffle = training[randomize]
training_tot_labels_shuffle = training_labels[randomize]
training_tot_choices_shuffle = training_choices[randomize]

training_shuffle = training_tot_shuffle[:180846]
validation_shuffle = training_tot_shuffle[-20094:]

training_labels_shuffle = training_tot_labels_shuffle[:180846]
validation_labels_shuffle = training_tot_labels_shuffle[-20094:]

training_choices_shuffle = training_tot_choices_shuffle[:180846]
validation_choices_shuffle = training_tot_choices_shuffle[-20094:]

print(training_shuffle.shape)
print(training_labels_shuffle.shape)
print(training_choices_shuffle.shape)
print(validation_shuffle.shape)
print(validation_labels_shuffle.shape)
print(validation_choices_shuffle.shape)


(180846, 128, 10)
(180846, 4)
(180846,)


# Keras Model

In [27]:
# LAYER DEFINITION

def DenseBlock(input,n):
  x = Dense(n)(input)
  x = BatchNormalization()(x)
  x = Activation('relu')(x)
  return x

input_Spect = Input(shape = [128, 10])
input_Label = Input(shape = [4,])

# First Branch - Encoder
m = Flatten(input_shape = [128, 10])(input_Spect)
m = DenseBlock(m, 128)
m = DenseBlock(m, 64)
m = DenseBlock(m, 32)
m = DenseBlock(m, 16)

# Second Branch - Conditioning Feed Forward Neural Network
x = Dense(16)(input_Label)
x = Activation('sigmoid')(x)
q = Dense(16)(input_Label)

# Encoded Input Conditioning
m = Multiply()([x,m])
encoded_input_conditioned = Add()([q, m]) # Input da passare al decoder

# Decoder
m = DenseBlock(encoded_input_conditioned, 128)
m = DenseBlock(m, 128)
m = DenseBlock(m, 128)
m = DenseBlock(m, 128)
m = Dense(128*10)(m)
m = Reshape((128,10),  input_shape=(128*10,))(m) # Output del modello

loss_tracker = keras.metrics.Mean(name="loss")
mse_metric = keras.metrics.MeanSquaredError(name="mse")

class CustomModel(tensorflow.keras.Model):
    @property
    def metrics(self):
        return [loss_tracker, mse_metric]

    def test_step(self, data):
        # Unpack the data
        x, y = data
        # Compute predictions
        y_pred = self([x[0],x[1]], training=False)
        # Indici match
        match = tf.where ( tf.equal(x[2][:], "match") )
        # Dati match
        data_match = K.gather(y, match)
        # Separazione dei dati PREDETTI sulla base degli indici relativi a match/non_match
        # Dati match
        pred_match = K.gather(y_pred, match)

        # Update metrica
        mse_metric.update_state(data_match, pred_match)

        return {"mse": mse_metric.result()}
    
    def train_step(self, data):
          # Unpack the data. Its structure depends on your model and on what you pass to `fit()`.
          x, y = data

          # Vettore C utilizzato per il calcolo della loss in caso di non_match
          C = 5 
          # Valore di probabilità utilizzato come peso
          ALPHA = 0.75 

          # Indici match
          match = tf.where ( tf.equal(x[2][:], "match") )

          # Indici non_match
          not_match = tf.where ( tf.equal(x[2][:], "non_match") )

          # Dati match
          data_match = K.gather(y, match)

          with tf.GradientTape() as tape:
              y_pred = self([x[0],x[1]], training=True)  # Forward pass

              # Separazione dei dati PREDETTI sulla base degli indici relativi a match/non_match
              # Dati match
              pred_match = K.gather(y_pred, match)
              # Dati non match
              pred_not_match = K.gather(y_pred, not_match) 

              loss_m = K.mean(keras.losses.mean_squared_error(data_match, pred_match)) + 1e-6  # Calcolo Loss Match
              loss_nm = K.mean(keras.losses.mean_squared_error(C,pred_not_match)) + 1e-6     # Calcolo Loss Non_Match

              loss = ALPHA * loss_m + (1 - ALPHA) * loss_nm     # loss utilizzata per l'update dei pesi

          # Compute gradients
          trainable_vars = self.trainable_variables
          gradients = tape.gradient(loss, trainable_vars)

          # Update weights
          self.optimizer.apply_gradients(zip(gradients, trainable_vars))

          # Compute our own metrics
          loss_tracker.update_state(loss)
          mse_metric.update_state(y, y_pred)
          return {"loss": loss_tracker.result(), "mse": mse_metric.result()}

def get_lr_metric(optimizer):
    def lr(y_true, y_pred):
        return optimizer._decayed_lr(tf.float32) # I use ._decayed_lr method instead of .lr
    return lr

opt = keras.optimizers.Adam(
    learning_rate = 0.00001,
    beta_1=0.95,
    beta_2=0.999
)

lr_metric = get_lr_metric(opt)
model = CustomModel(inputs=(input_Spect, input_Label), outputs = m)
model.compile(optimizer = opt, metrics=["mse", lr_metric])

In [28]:
history = model.fit([training_shuffle, training_labels_shuffle, training_choices_shuffle], 
          training_shuffle, 
          epochs=100, 
          batch_size=128, 
          validation_data=([validation_shuffle, validation_labels_shuffle, validation_choices_shuffle],validation_shuffle))

Epoch 1/100
1413/1413 [==============================] - 16s 10ms/step - loss: 4.8582 - mse: 0.3592 - val_mse: 0.2672
Epoch 2/100
1413/1413 [==============================] - 13s 10ms/step - loss: 1.6759 - mse: 2.3767 - val_mse: 0.3116
Epoch 3/100
1413/1413 [==============================] - 14s 10ms/step - loss: 0.8615 - mse: 4.0232 - val_mse: 0.3197
Epoch 4/100
1413/1413 [==============================] - 14s 10ms/step - loss: 0.6623 - mse: 4.4639 - val_mse: 0.2993
Epoch 5/100
1413/1413 [==============================] - 14s 10ms/step - loss: 0.5540 - mse: 4.6040 - val_mse: 0.2470
Epoch 6/100
1413/1413 [==============================] - 14s 10ms/step - loss: 0.4823 - mse: 4.6697 - val_mse: 0.2093
Epoch 7/100
1413/1413 [==============================] - 14s 10ms/step - loss: 0.4276 - mse: 4.7245 - val_mse: 0.2076
Epoch 8/100
1413/1413 [==============================] - 14s 10ms/step - loss: 0.3900 - mse: 4.7366 - val_mse: 0.1796
Epoch 9/100
1413/1413 [==============================] -

In [29]:
# Salvataggio del modello
model.save('/content/drive/MyDrive/models/21_02/model_21_02_4.h5')

In [30]:
# Salvataggio history di apprendimento
with open('/content/drive/MyDrive/models/21_02/trainHistoryDict_21_02_4', 'wb') as file_pi:
    pickle.dump(history.history, file_pi)

# Testing

In [31]:
def get_machine_id_list_for_test(target_dir,
                                 dir_name="test",
                                 ext="wav"):

    # create test files
    dir_path = os.path.abspath("{dir}/{dir_name}/*.{ext}".format(dir=target_dir, dir_name=dir_name, ext=ext))
    file_paths = sorted(glob.glob(dir_path))
    # extract id
    machine_id_list = sorted(list(set(itertools.chain.from_iterable(
        [re.findall('id_[0-9][0-9]', ext_id) for ext_id in file_paths]))))
    return machine_id_list

def test_file_list_generator(target_dir,
                             id_name,
                             dir_name="test",
                             prefix_normal="normal",
                             prefix_anomaly="anomaly",
                             ext="wav"):
  
    print("target_dir : {}".format(target_dir+"_"+id_name))

    normal_files = sorted(
    glob.glob("{dir}/{dir_name}/{prefix_normal}_{id_name}*.{ext}".format(dir=target_dir,
                                                                                 dir_name=dir_name,
                                                                                 prefix_normal=prefix_normal,
                                                                                 id_name=id_name,
                                                                                 ext=ext)))
    normal_labels = numpy.zeros(len(normal_files))
    anomaly_files = sorted(
    glob.glob("{dir}/{dir_name}/{prefix_anomaly}_{id_name}*.{ext}".format(dir=target_dir,
                                                                                  dir_name=dir_name,
                                                                                  prefix_anomaly=prefix_anomaly,
                                                                                  id_name=id_name,
                                                                                  ext=ext)))
    anomaly_labels = numpy.ones(len(anomaly_files))
    files = numpy.concatenate((normal_files, anomaly_files), axis=0)
    labels = numpy.concatenate((normal_labels, anomaly_labels), axis=0)
    print("test_file  num : {num}".format(num=len(files)))
    if len(files) == 0:
        print("no_wav_file!!")
    print("\n========================================")

    return files, labels

In [32]:
target_dir = "/content/drive/MyDrive/dcase/pump"

performance = []

machine_type = os.path.split(target_dir)[1]
print("============== MODEL LOAD ==============")
# set model path
model_file = "/content/drive/MyDrive/models/21_02/model_21_02_4.h5"

# load model file
if not os.path.exists(model_file):
  print("{} model not found ".format(machine_type))
  sys.exit(-1)
model = tensorflow.keras.models.load_model(model_file, custom_objects={'CustomModel': CustomModel, 'mse':mse_metric, 'lr': lr_metric})
# model.summary()

machine_id_list = get_machine_id_list_for_test(target_dir)

for id_str in machine_id_list:
  # load test file

  id_num = id_str.split("_")[1]

  # Definizione della label "match" da utilizzare in fase di testing e del min e max da utilizzare per la normalizzazione
  # i min e max sono stati calcolati a partire dai dati di training.
  if id_num == "00":
    match_labels = numpy.asarray([1,0,0,0])
    max = max_00
    min = min_00
  elif id_num == "02":
    match_labels = numpy.asarray([0,1,0,0])
    max = max_02
    min = min_02
  elif id_num == "04":
    match_labels = numpy.asarray([0,0,1,0])
    max = max_04
    min = min_04
  elif id_num == "06": 
    match_labels = numpy.asarray([0,0,0,1])
    max = max_06
    min = min_06

  test_files, y_true = test_file_list_generator(target_dir, id_str)
  #print("\n====== True Labels ======")
  #print(y_true)
  #print("==> ====== Match ID Labels ======")
  #print(match_labels.shape)
  #print("=================================\n")

  anomaly_score_list = []

  print("\n============== BEGIN TEST FOR A MACHINE ID {id} ==============".format(id=id_num))

  y_pred = [0. for k in test_files]

  for file_idx, file_path in tqdm(enumerate(test_files), total=len(test_files)):

    # Estrazione spettrogramma audio test
    data = file_to_vector_array(file_path, n_mels=N_MELS, n_fft=N_FFT, hop_length=HOP_LENGTH, power=POWER)

    # Normalizzazione spettrogramma di test
    data = ( data - min ) / ( max - min )

    # Estrazione delle frame 128x10
    data_splitted = numpy.zeros((60, 128, 10))
    index = 0
    i = 0
    while (i+5) < 303:
      vector_i = numpy.zeros((128,10))
      for j in range(0,128):
        vector_i[j] = data[j][i:i+10]
      data_splitted[index] = vector_i
      index += 1
      i = i+5

    # Calcolo dell'errore medio sulle frame estratte dallo spettrogramma
    elem_error = []
    for elem in data_splitted:
      predicted = model.predict([elem.reshape(1,128,10), match_labels.reshape(1,4)])

      errors = numpy.mean(numpy.square(elem - predicted), axis=1)
      elem_error.append(numpy.mean(errors))
    # Log dell'errore associato all'istanza di test
    y_pred[file_idx] = numpy.mean(elem_error)
    anomaly_score_list.append([os.path.basename(file_path), y_pred[file_idx]])
  # Calcolo AUC e pAUC per i dati con un certo ID_0x
  auc = metrics.roc_auc_score(y_true,y_pred)
  p_auc = metrics.roc_auc_score(y_true, y_pred, max_fpr=0.1)
  performance.append([auc, p_auc])
  print("AUC : {}".format(auc))
  print("pAUC : {}".format(p_auc))

  print("\n============ END OF TEST FOR A MACHINE ID ============")

# Stampa di AUC e pAUC medi su tutti i dati di test (media di AUC e pAUC sui vari ID).
print("\n============ AVERAGE PERFORMANCES ============")
averaged_performance = numpy.mean(numpy.array(performance, dtype=float), axis=0)
print(averaged_performance)


============== MODEL LOAD ==============


  0%|          | 0/243 [00:00<?, ?it/s]

target_dir : /content/drive/MyDrive/dcase/pump_id_00
test_file  num : 243


====== True Labels ======
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1.]
==> ====== Match ID Labels ======

============== BEGIN TEST FOR A MACHINE ID 00 ==============


  0%|          | 0/211 [00:00<?, ?it/s]

AUC : 0.841958041958042
pAUC : 0.7471475892528524

============ END OF TEST FOR A MACHINE ID ============
target_dir : /content/drive/MyDrive/dcase/pump_id_02
test_file  num : 211


====== True Labels ======
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
==> ====== Match ID Labels ======

============== BEGIN TEST FOR A MACHINE ID 02 ==============


...

AUC : 0.6035135135135135
pAUC : 0.6353722143195827

============ END OF TEST FOR A MACHINE ID ============
target_dir : /content/drive/MyDrive/dcase/pump_id_04
test_file  num : 200


====== True Labels ======
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1.]
==> ====== Match ID Labels ======

============== BEGIN TEST FOR A MACHINE ID 04 ==============


  0%|          | 0/202 [00:00<?, ?it/s]

AUC : 0.9983000000000001
pAUC : 0.9910526315789474

============ END OF TEST FOR A MACHINE ID ============
target_dir : /content/drive/MyDrive/dcase/pump_id_06
test_file  num : 202


====== True Labels ======
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
==> ====== Match ID Labels ======

============== BEGIN TEST FOR A MACHINE ID 06 ==============


100%|██████████| 202/202 [07:43<00:00,  2.29s/it]

AUC : 0.7883333333333333
pAUC : 0.6424148606811145

============ END OF TEST FOR A MACHINE ID ============

============ AVERAGE PERFORMANCES ============
[0.80802622 0.75399682]
